# Тематическая классификация длинных текстов - TFIDF и LogReg

In [ ]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:
 
!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
import sys; sys.path.append('./stepik-dl-nlp')

In [ ]:
import warnings
warnings.filterwarnings('ignore')
 
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score, confusion_matrix
 
import numpy as np
 
import matplotlib.pyplot as plt
%matplotlib inline
 
import collections
 
import torch
from torch import nn
from torch.nn import functional as F
 
import dlnlputils
from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
    vectorize_texts, SparseFeaturesDataset
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed
 
init_random_seed()
 
import random

## Предобработка текстов и подготовка признаков

In [ ]:
train_source = fetch_20newsgroups(subset='train')
test_source = fetch_20newsgroups(subset='test')
 
print('Количество обучающих текстов', len(train_source['data']))
print('Количество тестовых текстов', len(test_source['data']))
print()
print(train_source['data'][0].strip())
 
print()
print('Метка', train_source['target'][0])

### Подготовка признаков

In [ ]:
train_tokenized = tokenize_corpus(train_source['data'])
test_tokenized = tokenize_corpus(test_source['data'])
 
print(' '.join(train_tokenized[0]))

In [ ]:
MAX_DF = 0.8
MIN_COUNT = 5
vocabulary, word_doc_freq = build_vocabulary(train_tokenized, max_doc_freq=MAX_DF, min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

In [ ]:
plt.hist(word_doc_freq, bins=20)
plt.title('Распределение относительных частот слов')
plt.yscale('log');

In [ ]:
VECTORIZATION_MODE = 'tfidf'
train_vectors = vectorize_texts(train_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)
test_vectors = vectorize_texts(test_tokenized, vocabulary, word_doc_freq, mode=VECTORIZATION_MODE)

print('Размерность матрицы признаков обучающей выборки', train_vectors.shape)
print('Размерность матрицы признаков тестовой выборки', test_vectors.shape)
print()
print('Количество ненулевых элементов в обучающей выборке', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в обучающей выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

In [ ]:
plt.hist(train_vectors.data, bins=20)
plt.title('Распределение весов признаков')
plt.yscale('log');

### Распределение классов

In [ ]:
UNIQUE_LABELS_N = len(set(train_source['target']))
print('Количество уникальных меток', UNIQUE_LABELS_N)

In [ ]:
plt.hist(train_source['target'], bins=np.arange(0, 21))
plt.title('Распределение меток в обучающей выборке');

In [ ]:
plt.hist(test_source['target'], bins=np.arange(0, 21))
plt.title('Распределение меток в тестовой выборке');

### PyTorch Dataset

In [ ]:
train_dataset = SparseFeaturesDataset(train_vectors, train_source['target'])
test_dataset = SparseFeaturesDataset(test_vectors, test_source['target'])

## Обучение модели на PyTorch

In [ ]:
model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

best_val_loss, best_model = train_eval_loop(model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=F.cross_entropy,
                                            lr=1e-1,
                                            epoch_n=200,
                                            batch_size=32,
                                            l2_reg_alpha=0,
                                            lr_scheduler_ctor=scheduler)

## Оценка качества

In [ ]:
train_pred = predict_with_model(best_model, train_dataset)
 
train_loss = F.cross_entropy(torch.from_numpy(train_pred),
                             torch.from_numpy(train_source['target']).long())
 
print('Среднее значение функции потерь на обучении', float(train_loss))
print('Доля верных ответов', accuracy_score(train_source['target'], train_pred.argmax(-1)))
print('Доля ошибок', 1 - accuracy_score(train_source['target'], train_pred.argmax(-1)))
print()
8
 
 
test_pred = predict_with_model(best_model, test_dataset)
 
test_loss = F.cross_entropy(torch.from_numpy(test_pred),
                            torch.from_numpy(test_source['target']).long())
 
print('Среднее значение функции потерь на валидации', float(test_loss))
print('Доля верных ответов', accuracy_score(test_source['target'], test_pred.argmax(-1)))
print('Доля ошибок', 1 - accuracy_score(test_source['target'], test_pred.argmax(-1)))

# Альтернативная реализация на scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
                                                      max_df=MAX_DF,
                                                      min_df=MIN_COUNT)),
                             ('cls', LogisticRegression())))
sklearn_pipeline.fit(train_source['data'], train_source['target']);

## Оценка качества

In [ ]:
sklearn_train_pred = sklearn_pipeline.predict_proba(train_source['data'])
sklearn_train_loss = F.cross_entropy(torch.from_numpy(sklearn_train_pred),
                                                 torch.from_numpy(train_source['target']))
print('Среднее значение функции потерь на обучении', float(sklearn_train_loss))
print('Доля верных ответов', accuracy_score(train_source['target'], sklearn_train_pred.argmax(-1)))
print()
 
sklearn_test_pred = sklearn_pipeline.predict_proba(test_source['data'])
sklearn_test_loss = F.cross_entropy(torch.from_numpy(sklearn_test_pred),
                                                torch.from_numpy(test_source['target']))
print('Среднее значение функции потерь на валидации', float(sklearn_test_loss))
print('Доля верных ответов', accuracy_score(test_source['target'], sklearn_test_pred.argmax(-1)))

# Input Data Analysis

In [ ]:
def random_texts(source, label, n):
    """Choose n random texts with label from source."""
    texts = [source['data'][t] for t in range(len(source['data'])) if source['target'][t] == label]
    r_texts = random.sample(texts, n)
    return(r_texts)

In [ ]:
def print_random_texts(source, label, n):
    """Print n random texts with label from source."""
    texts = random_texts(source, label, n)
    print('Метка: {} ({})'.format(label, source['target_names'][label]))
    print()
    for text in texts:
        print(text)
        print('--------->\n')

In [ ]:
print_random_texts(train_source, 3, 5)

# Error analysis

## Confusion Matrix

In [ ]:
y_true = test_source['target']
y_pred = test_pred.argmax(-1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
print(cm)

In [ ]:
 def print_errors(source, label1, label2, n):
    """Print n random texts with true label1 and predicted label2 from source."""
    errors = [idx for idx in range(len(y_true)) if y_true[idx] == label1 and y_pred[idx] == label2]
    print('Верная метка: {} ({})'.format(label1, source['target_names'][label1]))
    print('Ошибочная метка: {} ({})'.format(label2, source['target_names'][label2]))
    n = random.sample(errors, n)
    print()
    for text in n:
        print(source['data'][text])
        print('--------->\n')

In [ ]:
 print_errors(test_source, 18, 16, 5)

In [ ]:
sum_by_true = np.sum(cm, axis=1)
correct = [cm[i, i] for i in range(UNIQUE_LABELS_N)]
errors_by_true = sum_by_true - correct
print('Error rate by true label:')
for i in range(UNIQUE_LABELS_N):
    print('{:.2%}'.format(errors_by_true[i]/sum_by_true[i]), i, test_source['target_names'][i])